In [ ]:
import IPython.display as ipd
import random 
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.utils import parallel_backend

import librosa
from librosa import display




import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *


%matplotlib inline

In [ ]:
data, sampling_rate = librosa.load("./data/Train/2022.wav")

In [ ]:
sampling_rate

In [ ]:


plt.figure(figsize=(12,4))
librosa.display.waveplot(data, sr=sampling_rate);

In [ ]:
train = pd.read_csv('./data/train.csv')

In [ ]:
train.head()

In [ ]:
i = random.choice(train.index)
data_dir = '.\\data\\'

audio_name = train.ID[i]
path = os.path.join(data_dir, 'Train', str(audio_name) + '.wav')

print('Class: ', train.Class[i])
x, sr = librosa.load("./data/Train/" + str(train.ID[i]) + ".wav")

plt.figure(figsize=(12,4))
librosa.display.waveplot(x, sr=sr);

In [ ]:
train['Class'].value_counts()/5434

In [ ]:
def parser(row):
       # function to load files and extract features
    file_name = os.path.join(os.path.abspath(data_dir), 'Train', str(row.ID) + '.wav')
    with parallel_backend('threading', n_jobs=-1):
           # handle exception to check if there isn't a file which is corrupted
        try:
              # here kaiser_fast is a technique used for faster extraction
            X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
              # we extract mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        except Exception as e:
            print("Error encountered while parsing file: ", file)
            return None, None

        feature = mfccs
        label = row.Class

        return [feature, label]

temp = train.apply(parser, axis=1)
# temp.columns = ['feature', 'label']

In [ ]:
df = pd.DataFrame(data=feature, columns=['feature'])

In [ ]:
df['label'] = label

In [ ]:
from sklearn.preprocessing import LabelEncoder

X = np.array(df['feature'].tolist())
y = np.array(df['label'].tolist())

lb = LabelEncoder()

y = keras.utils.to_categorical(lb.fit_transform(y))

In [ ]:
num_labels = y.shape[1]
filter_size = 2



In [ ]:
model = keras.models.Sequential()

model.add(layers.Dense(512, input_shape=(40,), 
                       activation='relu'))
model.add(layers.Dropout(0.1))


model.add(layers.Dense(256, 
                       activation='relu'))
model.add(layers.Dropout(0.1))


model.add(layers.Dense(num_labels, 
                       activation='softmax'))

model.compile(
    loss='categorical_crossentropy', 
    metrics=['accuracy'], 
    optimizer='adam')

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

Xtrain_ss = ss.fit_transform(Xtrain)
Xtest_ss = ss.transform(Xtest)

In [ ]:
model.fit(x=Xtrain, 
          y=ytrain, 
          batch_size=32, 
          epochs=5, 
          validation_data=(Xtest, ytest))